In [1]:
using Downloads, CSV, DataFrames
using Interpolations, PlotlyJS
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
total_cross_section_dir = parent_dir * "ImportedData\\tendl_total_neutron_sigma_data\\"
neutron_production_dir = parent_dir * "ImportedData\\my_tendl_neutron_production_data\\"
neutron_MTs_file = parent_dir * "ExportedData\\produce_only_neutrons_MTs.csv"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\produce_only_neutrons_MTs.csv"

In [2]:
#Import a dictionary of MTs to know what data to download
MT_df = CSV.read(neutron_MTs_file, DataFrame, stringtype=String)
MT_dict = Dict([])
rows, cols = size(MT_df)
for row in 1:rows
  MT_df[row, 2] = String(MT_df[row, 2][2:end-1])
  MTs = [parse(Int64, n) for n in split(MT_df[row, 2], ",")]
  for MT in MTs
      MT_dict[MT] = MT_df[row, 1] 
  end
end
MT_dict

Dict{Any, Any} with 10 entries:
  152 => "(0, -4)"
  875 => "(0, -1)"
  50  => "(0, -1)"
  16  => "(0, -1)"
  891 => "(0, -1)"
  876 => "(0, -1)"
  38  => "(0, -2)"
  21  => "(0, -1)"
  17  => "(0, -2)"
  37  => "(0, -3)"

In [3]:
nuclide_df_path = parent_dir * "ExportedData\\nuclide_list.csv"
nuclide_df = CSV.read(nuclide_df_path, DataFrame, stringtype=String)
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
nuclide_list = [get_mass_name(nuclide) for nuclide in nuclide_df[!,1]]

5842-element Vector{Tuple{Int64, Any}}:
 (1, 'H')
 (2, 'H')
 (3, 'H')
 (3, "He")
 (3, "Li")
 (4, 'H')
 (4, "He")
 (4, "Li")
 (5, 'H')
 (5, "He")
 (5, "Li")
 (5, "Be")
 (6, 'H')
 ⋮
 (291, "Lv")
 (291, "Ts")
 (292, "Mc")
 (292, "Lv")
 (292, "Ts")
 (293, "Lv")
 (293, "Lv")
 (293, "Ts")
 (293, "Og")
 (294, "Ts")
 (294, "Og")
 (295, "Og")

In [4]:
function get_file_path(parent_dir, mt, mass, elem)
    mt_str, mass_str = lpad(string(mt), 3, "0"), lpad(string(mass), 3, "0")
    file_path = parent_dir * "n-" * elem * mass_str * "-" * "MT" * mt_str * ".csv"
    return file_path
end
function get_file_path(parent_dir, mt, nuclide)
    mass, elem = get_mass_name(nuclide)
    mt_str, mass_str = lpad(string(mt), 3, "0"), lpad(string(mass), 3, "0")
    file_path = parent_dir * "n-" * elem * mass_str * "-" * "MT" * mt_str * ".csv"
    return file_path
end
get_file_path(total_cross_section_dir, 1, nuclide_df[1,1])

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\tendl_total_neutron_sigma_data\\n-H001-MT001.csv"

In [5]:
function get_total_neutron_cross_section(nuclide)
    file_path = get_file_path(total_cross_section_dir, 1, nuclide)
    return CSV.read(file_path, DataFrame)
end
function get_total_neutron_cross_section(mass, elem)
    file_path = get_file_path(total_cross_section_dir, 1, mass, elem)
    return CSV.read(file_path, DataFrame)
end
get_total_neutron_cross_section(nuclide_list[1][1], nuclide_list[1][2])

,E(MeV),xs(mb)
,Float64,Float64
1,1.0e-11,1.1778e6
2,1.03223e-11,1.15927e6
3,1.06549e-11,1.14104e6
4,1.09984e-11,1.12309e6
5,1.13527e-11,1.10542e6
6,1.17186e-11,1.08803e6
7,1.20962e-11,1.07092e6
8,1.24861e-11,1.05408e6
9,1.28884e-11,1.0375e6


In [6]:
neutron_production_file_names = readdir(neutron_production_dir)
file_path = neutron_production_dir * neutron_production_file_names[2]
function neutrons_produced(file_path)
    MT = tryparse(Int64, split(split(file_path, "MT")[end], ".")[1])
    neutrons = tryparse(Int64, split(MT_dict[MT], ",")[end][1:end-1]) * -1
    return neutrons
end
neutrons_produced(file_path), file_path

(2, "C:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\my_tendl_neutron_production_data\\n-Ac227-MT017.csv")

In [7]:
function get_neutron_production_cross_sections(nuclide)
    mass, elem = get_mass_name(nuclide)
    mass_str = lpad(string(mass), 3, "0")
    file_paths = [neutron_production_dir * file_name 
                for file_name in neutron_production_file_names 
                if occursin(elem * mass_str, file_name)]
    return [(CSV.read(file_path, DataFrame), neutrons_produced(file_path)) 
            for file_path in file_paths]
end
function get_neutron_production_cross_sections(mass, elem)
    mass_str = lpad(string(mass), 3, "0")
    file_paths = [neutron_production_dir * file_name 
                for file_name in neutron_production_file_names 
                if occursin(elem * mass_str, file_name)]
    return [(CSV.read(file_path, DataFrame), neutrons_produced(file_path)) 
            for file_path in file_paths]
end
get_neutron_production_cross_sections("2H")

1-element Vector{Tuple{DataFrame, Int64}}:
 (124×2 DataFrame
 Row │ E(MeV)   xs(mb)   
     │ Float64  Float64  
─────┼───────────────────
   1 │   3.339   0.0
   2 │   3.4     1.2
   3 │   3.5     2.9
   4 │   3.75    7.10318
   5 │   4.0    13.5
   6 │   4.25   19.0312
   7 │   4.5    25.0
   8 │   5.0    37.0
   9 │   5.5    48.6414
  10 │   5.6    51.0236
  11 │   6.0    60.2
  ⋮  │    ⋮        ⋮
 115 │ 132.0    69.4702
 116 │ 134.0    68.8945
 117 │ 136.0    68.3321
 118 │ 138.0    67.7825
 119 │ 140.0    67.2451
 120 │ 142.0    66.7194
 121 │ 144.0    66.2049
 122 │ 146.0    65.7011
 123 │ 148.0    65.2076
 124 │ 150.0    64.724
         103 rows omitted, 1)

In [8]:
get_total_neutron_cross_section("2H")

,E(MeV),xs(mb)
,Float64,Float64
1,1.0e-11,136398.0
2,1.03223e-11,134252.0
3,1.06549e-11,132142.0
4,1.09984e-11,130063.0
5,1.13527e-11,128019.0
6,1.17186e-11,126005.0
7,1.20962e-11,124024.0
8,1.24861e-11,122074.0
9,1.28884e-11,120155.0


The match_energies function returns the total cross section for a projectile interacting with a target corresponding to some given energy (energy_rxn)

energy is an individial energy value for which a reaction cross section is measured.  

total_energies is an array of energy values for which a total cross section of some projectile and target was measured. 

interpolation is a function that can be used to calculate the total cross section for a projectile corresponding to the same energy value as the input named energy. 


Eg: 

corresponding_total_σ = [match_energies(energy, total_energy, itp, total_cross_sections) for energy in energies_rxn]

In [9]:
function match_energies(energy_rxn, total_energies, interpolation,
    total_cross_sections)
    #itp = Interpolations.interpolate(x, y, FritschCarlsonMonotonicInterpolation())
    try 
            index = findfirst(x -> x ==(energy_rxn), total_energies)
            return total_cross_sections[index] 
    catch 
            return interpolation.(energy_rxn)
    end
end

match_energies (generic function with 1 method)

In [10]:
x_axis, y_axis =  "E(MeV)", "xs(mb)"
df_total = get_total_neutron_cross_section("2H")
df, neutrons = get_neutron_production_cross_sections("2H")[1]
trace = [scatter(x  = df_total[!,x_axis], y  = df_total[!,y_axis], mode="markers",
                name = "total"),
        scatter(x  = df[!,x_axis], y  = df[!,y_axis], mode="markers", 
        name = ("produce " * string(neutrons) * "n"))]
plot(trace, Layout(xaxis_title = x_axis, yaxis_title = y_axis, 
                legend_title_text="Legend", yaxis_type = "log", xaxis_type = "log"))

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

Now to calculate neutrons/MeV

In [11]:
function find_max_neutrons_per_energy(energies_rxn, total_energy, 
                                        total_σ, reaction_σ, neutrons)
    itp = Interpolations.interpolate(total_energy, total_σ, 
                    FritschCarlsonMonotonicInterpolation())
    corresponding_total_σ = [match_energies(energy, total_energy, 
                                itp, total_σ) for energy in energies_rxn]
    reaction_per_energy = reaction_σ ./ (energies_rxn .*  corresponding_total_σ)
    best_index = argmax(reaction_per_energy)
    neutrons_per_energy = neutrons * reaction_per_energy[best_index]
    return (neutrons_per_energy, energies_rxn[best_index], 
            corresponding_total_σ[best_index], reaction_σ[best_index])
end

find_max_neutrons_per_energy (generic function with 1 method)

In [12]:
find_max_neutrons_per_energy(df[!, "E(MeV)"], df_total[!, "E(MeV)"],
                            df_total[!, "xs(mb)"], df[!, "xs(mb)"], neutrons)

(0.014687340569339187, 14.5, 788.636, 167.953)

In [13]:
a = [(x, x+1) for x in 1:10]
[(at, b) for (at, b) in a]

10-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 6)
 (6, 7)
 (7, 8)
 (8, 9)
 (9, 10)
 (10, 11)

In [25]:
function find_max_neutrons_per_energy(nuclide)
    try
        df_total = get_total_neutron_cross_section(nuclide)
        neutron_tuples = get_neutron_production_cross_sections(nuclide)
        max_neutrons_per_energy_list = [find_max_neutrons_per_energy(df[!, "E(MeV)"], 
                                        df_total[!, "E(MeV)"], df_total[!, "xs(mb)"], 
                                        df[!, "xs(mb)"], neutrons) 
                                        for (df, neutrons) in neutron_tuples]
        max_neutrons_per_energy_filtered = [output[1] for output in 
                                            max_neutrons_per_energy_list]
        best_index = argmax(max_neutrons_per_energy_filtered)
        return max_neutrons_per_energy_list[best_index]
    catch 
        err_msg = "did not work for " * nuclide
        print(err_msg)
        return [0]
    end
end
find_max_neutrons_per_energy("227Ac")

(0.03829420926737219, 10.5, 5290.07, 2127.08)

Yay it works :) Now to loop through every nuclide :) 

In [34]:
nuclide_df[!, "Neutrons per MeV"] = [find_max_neutrons_per_energy(nuclide)[1]
                                    for nuclide in nuclide_df[!,1]]

did not work for 1hdid not work for 3hedid not work for 3lidid not work for 4hdid not work for 4hedid not work for 4lidid not work for 5hdid not work for 5hedid not work for 5lidid not work for 5bedid not work for 6hdid not work for 6hedid not work for 6lidid not work for 6lidid not work for 6bedid not work for 6bdid not work for 7hdid not work for 7hedid not work for 7bedid not work for 7bedid not work for 7bdid not work for 8hedid not work for 8lidid not work for 8lidid not work for 8bedid not work for 8bedid not work for 8bedid not work for 8bdid not work for 8bdid not work for 8cdid not work for 9hedid not work for 9lidid not work for 9bdid not work for 9bdid not work for 9cdid not work for 10hedid not work for 10lidid not work for 10lidid not work for 10lidid not work for 10bdid not work for 10bdid not work for 10cdid not work for 10ndid not work for 11lidid not work for 11bedid not work for 11bedid not work for 11cdid not work for 11cdid not work for 11ndid not work for 11ndid no

5842-element Vector{Real}:
 0
 0.014687340569339187
 0.0037757153182568417
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [35]:
nuclide_df

,1n,Neutrons per MeV
,String,Real
1,1h,0
2,2h,0.0146873
3,3h,0.00377572
4,3he,0
5,3li,0
6,4h,0
7,4he,0
8,4li,0
9,5h,0


In [55]:
amu_df[!,"Binding energy per nucleon (keV)"]
function convert_to_MeV(energy)
    try 
        return parse(Float64, energy) / Rational(1000)
    catch
        try 
            return parse(Float64, energy[1:end-1]) / Rational(1000)
        catch
            return energy #should be "missing"
        end
    end
end

convert_to_MeV (generic function with 1 method)

In [65]:
amu_df = CSV.read(parent_dir * "ImportedData\\Atomic_mass_table_2020.csv", DataFrame, 
        stringtype = String)
amu_df[!,"A"] = [Int64(a) for a in amu_df[!,"A"]]
amu_df[!,"Binding energy per nucleon (MeV)"] = [convert_to_MeV(energy) for energy in 
                                                amu_df[!,"Binding energy per nucleon (keV)"]]
amu_df[!, "Binding energy per nucleon (MeV)"] 

3558-element Vector{Float64}:
  0.0
  0.0
  1.1122831000000002
  2.8272654000000004
  2.57268044
 -2.267
  1.7204491
  7.0739156
  1.1537602999999998
  1.3363592000000002
  5.5121325
  5.266132499999999
  0.018
  ⋮
  7.129
  7.116
  7.098
  7.124
  7.116
  7.095
  7.111
  7.095
  7.078
  7.092
  7.079
  7.076

In [58]:
names(amu_df)

16-element Vector{String}:
 "Column1"
 "N"
 "Z"
 "A"
 "Elt."
 "Orig."
 "Mass excess (keV)"
 "Mass excess (uncertainty)"
 "Binding energy per nucleon (keV)"
 "Binding energy per nucleon (uncertainty)"
 "Beta-decay Type"
 "Beta-decay energy (keV)"
 "Beta-decay energy (uncertainty)"
 "Atomic mass (μu)"
 "Atomic mass (uncertainty)"
 "Binding energy per nucleon (MeV)"

In [70]:
amu_df[!, "A"], amu_df[!, "Elt."]

([1, 1, 2, 3, 3, 3, 4, 4, 4, 5  …  291, 292, 292, 292, 293, 293, 293, 294, 294, 295], ["n", "H", "H", "H", "He", "Li", "H", "He", "Li", "H"  …  "Ts", "Mc", "Lv", "Ts", "Lv", "Ts", "Og", "Ts", "Og", "Og"])

In [86]:
amu_df[!, "A Elt."] = [(string(amu_df[row, "A"]) * amu_df[row, "Elt."]) 
                        for row in 1:size(amu_df)[1]]

3558-element Vector{String}:
 "1n"
 "1H"
 "2H"
 "3H"
 "3He"
 "3Li"
 "4H"
 "4He"
 "4Li"
 "5H"
 "5He"
 "5Li"
 "5Be"
 ⋮
 "291Mc"
 "291Lv"
 "291Ts"
 "292Mc"
 "292Lv"
 "292Ts"
 "293Lv"
 "293Ts"
 "293Og"
 "294Ts"
 "294Og"
 "295Og"

In [108]:
function glue_mass_name(nuclide)
    mass, name = get_mass_name(nuclide)
    return string(mass) * name
end
find_row(x) = [row for row in 1:size(amu_df)[1] if amu_df[row, "A Elt."] == x][1]
find_row(glue_mass_name("2h"))

3